In [10]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import Imputer, VectorAssembler
from pyspark.ml.regression import LinearRegression

In [2]:
spark = SparkSession.builder.appName('Test').getOrCreate()

22/12/21 11:20:39 WARN Utils: Your hostname, codespaces-e40fbf resolves to a loopback address: 127.0.0.1; using 172.16.5.4 instead (on interface eth0)
22/12/21 11:20:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/21 11:20:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
spark

In [4]:
df = spark.read.csv('tips.csv', header=True, inferSchema=True)
df

DataFrame[total_bill: double, tip: double, sex: string, smoker: string, day: string, time: string, size: int]

In [5]:
df.printSchema()
df.show(1)

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 1 row



In [6]:
type(df)
df.columns
df.select(['total_bill', 'tip']).show(1)
# df[['total_bill', 'tip']].show(1)
df.dtypes
df.describe().show()

+----------+----+
|total_bill| tip|
+----------+----+
|     16.99|1.01|
+----------+----+
only showing top 1 row



+-------+------------------+------------------+------+------+----+------+------------------+
|summary|        total_bill|               tip|   sex|smoker| day|  time|              size|
+-------+------------------+------------------+------+------+----+------+------------------+
|  count|               244|               244|   244|   244| 244|   244|               244|
|   mean|19.785942622950824|2.9982786885245902|  null|  null|null|  null| 2.569672131147541|
| stddev| 8.902411954856857|1.3836381890011815|  null|  null|null|  null|0.9510998047322347|
|    min|              3.07|               1.0|Female|    No| Fri|Dinner|                 1|
|    max|             50.81|              10.0|  Male|   Yes|Thur| Lunch|                 6|
+-------+------------------+------------------+------+------+----+------+------------------+



In [7]:
df = df.withColumn('doubesize', df['size']*2)

In [8]:
df.show(1)

+----------+----+------+------+---+------+----+---------+
|total_bill| tip|   sex|smoker|day|  time|size|doubesize|
+----------+----+------+------+---+------+----+---------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        4|
+----------+----+------+------+---+------+----+---------+
only showing top 1 row



In [9]:
df = df.drop('doubesize')
df.show(1)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 1 row



In [10]:
df.withColumnRenamed('day', 'day_in_week').show(1)

+----------+----+------+------+-----------+------+----+
|total_bill| tip|   sex|smoker|day_in_week|  time|size|
+----------+----+------+------+-----------+------+----+
|     16.99|1.01|Female|    No|        Sun|Dinner|   2|
+----------+----+------+------+-----------+------+----+
only showing top 1 row



In [15]:
df.na.drop(thresh=1, subset=['tip']).show(1)
df.na.fill('Missing Values', ['tip', 'sex']).show(1)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 1 row

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 1 row



In [18]:
imputer = Imputer(
    inputCols=['tip', 'size'],
    outputCols=[f'{col}_imputed' for col in ['tips', 'size']]
).setStrategy('mean')

imputer.fit(df).transform(df).show(1)

+----------+----+------+------+---+------+----+------------+------------+
|total_bill| tip|   sex|smoker|day|  time|size|tips_imputed|size_imputed|
+----------+----+------+------+---+------+----+------------+------------+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|        1.01|           2|
+----------+----+------+------+---+------+----+------------+------------+
only showing top 1 row



In [28]:
df.filter("size>=3").count()
df.filter((df['size']>=3) & (df['tip']>2)).count()
df.filter((df['size']>=3) | 
          (df['tip']>2)).count()
df.filter(~(df['size']>=3)).show(1)

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
+----------+----+------+------+---+------+----+
only showing top 1 row



In [32]:
df.groupBy('sex').sum().show()

+------+------------------+-----------------+---------+
|   sex|   sum(total_bill)|         sum(tip)|sum(size)|
+------+------------------+-----------------+---------+
|Female|1570.9499999999998|           246.51|      214|
|  Male|3256.8200000000024|485.0700000000001|      413|
+------+------------------+-----------------+---------+



In [9]:
featureassembler = VectorAssembler(inputCols=['total_bill', 'size'], outputCol='Independent Feature')
output = featureassembler.transform(df)
final_data = output.select('Independent Feature', 'tip')
final_data.show(1)

+-------------------+----+
|Independent Feature| tip|
+-------------------+----+
|        [16.99,2.0]|1.01|
+-------------------+----+
only showing top 1 row



In [11]:
train_data, test_data = final_data.randomSplit([0.75, 0.25])
regressor = LinearRegression(featuresCol='Independent Feature', labelCol='tip')
regressor = regressor.fit(train_data)
regressor.coefficients

22/12/21 11:27:19 WARN Instrumentation: [f4bae713] regParam is zero, which might cause numerical instability and overfitting.
22/12/21 11:27:19 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS
22/12/21 11:27:19 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
22/12/21 11:27:19 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS
22/12/21 11:27:19 WARN InstanceBuilder$NativeLAPACK: Failed to load implementation from:dev.ludovic.netlib.lapack.JNILAPACK


DenseVector([0.0977, 0.1832])

In [12]:
regressor.intercept

0.6260877438140336

In [14]:
pred_results = regressor.evaluate(test_data)
pred_results.predictions.show(5)

+-------------------+----+------------------+
|Independent Feature| tip|        prediction|
+-------------------+----+------------------+
|         [7.56,2.0]|1.44| 1.731070250426483|
|         [7.74,2.0]|1.44|1.7486539632508138|
|         [9.55,2.0]|1.45|1.9254679644288086|
|         [9.78,2.0]|1.73|1.9479360419265643|
|        [10.33,3.0]|1.67| 2.184897337329517|
+-------------------+----+------------------+
only showing top 5 rows



In [15]:
pred_results.meanAbsoluteError

0.7541978826707247